In [1]:
import os,sys,tqdm
import numpy as np
import tensorflow as tf
import tensorflow.keras
from tensorflow.keras.datasets import *
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing.text import *

from collections import Counter
import pandas as pd
import shutil
import pickle
import gc
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
# The GPU id to use, usually either "0" or "1"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [2]:
def basic_statistics(all_length):
    '''
    input: length list of elements e.g.[1,1,1,3,5,9,4,2,1,3,54,78,5...]
    output1: mean、std、mode、min、q1、median(q2)、q3、max、iqr、outlier、far out
    output2: statistics graph、10%~90% form
    '''
    stat_dict = {}
    stat_dict['mean'] = np.mean(all_length)
    stat_dict['std'] = np.std(all_length)
    stat_dict['mode'] = np.argmax(np.bincount(all_length))
    stat_dict['min'] = np.min(all_length)
    stat_dict['q1'] = np.quantile(all_length,0.25)
    stat_dict['median'] = np.quantile(all_length,0.5)
    stat_dict['q3'] = np.quantile(all_length,0.75)
    stat_dict['max'] = np.max(all_length)
    stat_dict['iqr'] = stat_dict['q3'] - stat_dict['q1']
    stat_dict['outlier'] = stat_dict['q3'] + 1.5*stat_dict['iqr']
    stat_dict['far_out'] = stat_dict['q3'] + 3*stat_dict['iqr']
    for i in [10,20,30,40,50,60,70,80,90,100]:
        stat_dict[str(i)+'%'] = np.percentile(all_length,i)
    return pd.DataFrame.from_dict(stat_dict,orient='index',columns=['length'])

In [2]:
max_words = 8352#8352 #Top most frequent words to consider. Any less frequent word will appear as oov_char value in the sequence data.
max_length = 360#360

In [3]:
word_index = reuters.get_word_index()
print('all_words#:',len(word_index))
(x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=max_words,maxlen=max_length,
                                                         test_split=0.2,seed=830913)
print(len(x_train), 'train sequences')
print(len(x_test), 'test sequences')
word_index

all_words#: 30979
8260 train sequences
2066 test sequences


{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

In [13]:
# train_len = [len(x) for x in x_train]
# test_len = [len(x) for x in x_test]
# all_len = train_len
# all_len.extend(test_len)
# basic_statistics(all_len)

,length
mean,145.964197
std,145.878476
mode,17.000000
min,2.000000
q1,60.000000
median,95.000000
q3,180.000000
max,2376.000000
iqr,120.000000
outlier,360.000000


In [15]:
# df = pd.DataFrame(all_len)
# df.to_excel('./results/length_dist.xlsx', header=False, index=False)
# df

In [21]:
# train_words = []
# for x in x_train:
#     train_words.extend(x)
# test_words = []
# for x in x_test:
#     test_words.extend(x)
# all_words = train_words
# all_words.extend(test_words)
# all_statistcs = Counter(all_words)
# all_statistcs

Counter({1: 11228,
         53: 4213,
         352: 647,
         26: 8451,
         14: 15015,
         279: 801,
         39: 5818,
         72: 3091,
         4497: 26,
         18: 11039,
         83: 2597,
         5291: 21,
         88: 2381,
         5397: 20,
         11: 20141,
         3412: 37,
         19: 10755,
         151: 1363,
         230: 962,
         831: 253,
         15: 13329,
         165: 1232,
         318: 707,
         3780: 33,
         124: 1676,
         1527: 117,
         1424: 128,
         35: 6588,
         5302: 20,
         12: 16668,
         17: 11191,
         486: 459,
         341: 663,
         142: 1466,
         255: 870,
         219: 997,
         429: 528,
         68: 3363,
         146: 1402,
         252: 882,
         191: 1098,
         15448: 3,
         3631: 35,
         2283: 65,
         71: 3120,
         10: 29581,
         342: 660,
         49: 4565,
         1977: 80,
         324: 695,
         27: 8311,
         9222: 

In [28]:
# df = pd.DataFrame.from_dict(dict(all_statistcs), orient = 'index')
# df.to_excel('./results/words_dist2.xlsx', header=False, index=True)
# df

,0
1,11228
53,4213
352,647
26,8451
14,15015
...,...
24452,1
18567,2
27222,1
26864,1


In [4]:
trainX = tf.keras.preprocessing.sequence.pad_sequences(x_train,maxlen=max_length,padding='post',value=0)
testX = tf.keras.preprocessing.sequence.pad_sequences(x_test,maxlen=max_length,padding='post',value=0)
print(trainX.shape, testX.shape)

(8260, 360) (2066, 360)


In [5]:
hidden_dim = 128
# do = 0.1

## Graph execution
### Embedder

In [42]:
int_id = Input(shape=(max_length,), dtype='int32', name='int_ids') # 輸入的api funvtion name ID
int_ids = Masking(mask_value=0)(int_id)
sent_emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')(int_ids) 

### Encoder

In [43]:
rnn = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)(sent_emb)
rnn = BatchNormalization(name='bn')(rnn)


### Filter

In [44]:
fil = TimeDistributed(Dense(1,activation='sigmoid',
                             name='filter_out'),name='TD2')(rnn)

### Classfier

In [47]:
mul = Multiply()([fil,sent_emb])
clf = LSTM(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm')(mul)
clf = BatchNormalization(name='bn3')(clf)
clf = Dense(max(y_train)+1,activation='softmax',name='clf')(clf)

## Compile

In [48]:
model = Model(inputs=int_id, outputs = clf)
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
int_ids (InputLayer)            [(None, 358)]        0                                            
__________________________________________________________________________________________________
masking_1 (Masking)             (None, 358)          0           int_ids[0][0]                    
__________________________________________________________________________________________________
glove_emb (Embedding)           (None, 358, 128)     1280000     masking_1[0][0]                  
__________________________________________________________________________________________________
common_extract (GRU)            (None, 358, 64)      37248       glove_emb[0][0]                  
______________________________________________________________________________________________

In [53]:
# loss
import keras.backend as K
def custom_objective(layer):
    return K.sum(layer.output)
#     return K.sum(layer.output)
# kk = tf.keras.backend.ea
model.compile(loss=custom_objective(model.get_layer(name='TD2')),optimizer='adam')

Using TensorFlow backend.


OperatorNotAllowedInGraphError: using a `tf.Tensor` as a Python `bool` is not allowed in Graph execution. Use Eager execution or decorate this function with @tf.function.

## Eager Execution

In [52]:
# whole model
do = 0
init = tensorflow.keras.initializers.Ones()
class base_model(Model):
    def __init__(self):
        super(base_model, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/2),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.bn1 = BatchNormalization(name='bn1')
        self.fil = Dense(1,activation='hard_sigmoid',kernel_initializer=init,bias_initializer=init,name='filter_out')
        #self.fil = TimeDistributed(Dense(1,activation='sigmoid', name='filter_out'),name='TD2')
        self.mul = Multiply()
        self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = LSTM(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')
    def transform(self,x):
        return tf.math.round(x)
    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.rnn1(x1)
        x = self.bn1(x)
        y = self.fil(x)
        y1 = self.transform(y)
        x2 = self.mul([y1,x1])
        x = self.rnn2(x2) #x
        x = self.bn2(x)
        y2 = self.out(x)
        return y,y1,y2
        #return y,y1,y2,x2
        
model = base_model()

In [91]:
# partial1 model
# init_w = tensorflow.keras.initializers.Constant(value=1.667) #portyion=0.6, w=0.9, b = 0.8-0.85 (0.83從0開始)
init_w = tensorflow.keras.initializers.RandomNormal(mean=1.8,stddev=0.333)
# init_b = tensorflow.keras.initializers.Constant(value=1.667) #w=1 ; b=0.499, portion=1
init_b = tensorflow.keras.initializers.RandomNormal(mean=1.667,stddev=0.333)
def onezero(x):
    portion = 0.6#0.6#0.6 #0.6~1
    z = tf.where(x>=1.0, x - x + 1.0, x)
    y = tf.where(z<=0.0, z - z + 0.0, portion*z)
    return y

class base_model_1(Model):
    def __init__(self):
        super(base_model_1, self).__init__()
        self.mask = Masking(mask_value=0)
        self.emb = Embedding(max_words, hidden_dim,input_length=max_length
                    ,trainable=True,name='glove_emb')
        self.rnn1 = GRU(int(hidden_dim/4),return_sequences=True,return_state=False,name='common_extract'
                      ,trainable=True)
        self.att = Attention(name='selfatt')
        self.bn1 = BatchNormalization(name='bn1')
        #self.fil = Dense(1,activation=onezero,name='filter_out')
        self.fil = TimeDistributed(Dense(1,activation=onezero,kernel_initializer=init_w,bias_initializer=init_b, name='filter_out'),name='TD2') #relu/linear/step function

    def call(self,x):
        x = self.mask(x)
        x1 = self.emb(x)
        x = self.att([x1,x1])
        #x = self.rnn1(x1)
        #x = self.bn1(x)
        y = self.fil(x)
        return x1,y

model1 = base_model_1()
#phase2
# model3 = load_model(saveP)

# model3 = base_model_1()
# model3.load_weights('./model/2019110501/model1')#,by_name=True)
# model1.emb.set_weights(model3.emb.get_weights())

In [92]:
# partial2 model
class base_model_2(Model):
    def __init__(self):
        super(base_model_2, self).__init__()
        self.mul = Multiply()
        #self.rnn2 = Bidirectional(GRU(int(hidden_dim/2),dropout=do,recurrent_dropout=do,name='lstm'))
        self.rnn3 = GRU(int(hidden_dim/2))
        self.bn2 = BatchNormalization(name='bn2')
        self.out = Dense(max(y_train)+1,activation='softmax',name='clf')

    def call(self,x1,y1):
        x2 = self.mul([y1,x1])
        x = self.rnn3(x2) #x2 #y1=weight|binary
        x = self.bn2(x)
        y2 = self.out(x)
        return y2
    
model2 = base_model_2()

In [448]:
'''
# TEST
x = tf.random.uniform((32, 6))
out1,out2 = model1(x)
out2 = tf.squeeze(out2,axis=-1)
# out2 = out2.astype('float32')
# out2 = tf.dtypes.cast(out2, tf.float8)
# print(out2.shape)
yy = tf.matmul(out2,kk)
print(yy[0].shape)
tf.where(yy[0]==0,1,0)

gg = tf.range(-5,5)
gg = tf.expand_dims(gg,axis=0)
gg = tf.keras.backend.repeat_elements(gg,rep=32,axis=0)
tf.where(gg==1,gg,0)

kk = tf.Variable(np.array([[1.0,1.0,1.0,0.0,0.0,0.0],[0.0,1.0,1.0,1.0,0.0,0.0],[0.0,0.0,1.0,1.0,1.0,0.0],
                           [0.0,0.0,0.0,1.0,1.0,1.0]]).T,dtype='float32')
kk = tf.expand_dims(kk,axis=0)
kk = tf.keras.backend.repeat_elements(kk,rep=1,axis=0)
kk.shape
'''

(32, 4)


<tf.Tensor: id=5770753, shape=(32, 4), dtype=int32, numpy=
array([[1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1],
       [1, 1, 1, 1]], dtype=int32)>

In [93]:
# 記得要跑到一個好的data
# batch_size = 128 #,reshuffle_each_iteration=True
train_ds = tf.data.Dataset.from_tensor_slices((trainX,y_train)).shuffle(trainX.shape[0]).batch(batch_size)
valid_ds = tf.data.Dataset.from_tensor_slices((testX,y_test)).batch(batch_size)

In [94]:
seq_num = 2 #連續幾個才叫做連續，要改modify_idx看有幾個

arr_len = max_length - seq_num + 1
seq_arr = []
for i in range(arr_len):
    ori_np = np.array([0]*max_length)
    modify_idx = [i,i+1] #要跟著seq_num改
    ori_np[modify_idx]=1
    seq_arr.append(ori_np)
seq_arr = np.array(seq_arr)
seq_mask = tf.Variable(seq_arr.T,dtype='float32')
seq_mask = tf.expand_dims(seq_mask,axis=0)
seq_mask

<tf.Tensor: id=3143839, shape=(1, 360, 359), dtype=float32, numpy=
array([[[1., 0., 0., ..., 0., 0., 0.],
        [1., 1., 0., ..., 0., 0., 0.],
        [0., 1., 1., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 1., 1., 0.],
        [0., 0., 0., ..., 0., 1., 1.],
        [0., 0., 0., ..., 0., 0., 1.]]], dtype=float32)>

In [95]:
def loss_object1(predictions): #filter loss
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def one_percentage(predictions): #1 num
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    loss_ = tf.reduce_mean(predictions)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def seq_loss(predictions):
    mask = tf.math.logical_not(tf.math.equal(predictions, 0))
    predictions = tf.squeeze(predictions,axis=-1)
    results = tf.matmul(predictions,seq_mask)
    results = tf.where(results==seq_num,1.0,0.0)
    loss_ = tf.reduce_mean(results)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)
def word_reproduce1(prediction_int):
    return prediction_int
def word_reproduce2(prediction_int):
    return prediction_int
loss_object2 = tf.keras.losses.SparseCategoricalCrossentropy() #clf loss

optimizer1 = tf.keras.optimizers.Adam(learning_rate=0.0003)
optimizer2 = tf.keras.optimizers.RMSprop()

train_loss = tf.keras.metrics.Mean(name='train_loss') #total_loss
train_accloss = tf.keras.metrics.Mean(name='train_accloss')#loss_acc
train_filloss = tf.keras.metrics.Mean(name='train_filloss') #loss_filter
train_seqloss = tf.keras.metrics.Mean(name='train_seqloss') #loss_seq
train_ones = tf.keras.metrics.Mean(name='train_ones') #ones_num
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy') #acc_rate

test_loss = tf.keras.metrics.Mean(name='test_loss') #total_loss
test_accloss = tf.keras.metrics.Mean(name='test_accloss')#loss_acc
test_filloss = tf.keras.metrics.Mean(name='test_filloss') #loss_filter
test_seqloss = tf.keras.metrics.Mean(name='test_seqloss')
test_ones = tf.keras.metrics.Mean(name='test_ones') #ones_num
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [96]:
# seperate partial model
alpha = 0.000002 #pahse1: -0.1 / 0.0 ; phase2: 0.01~0.05~0.1 有1-就是希望1越多，沒1-就是希望0越多1越少
beta = 1.0 #clf loss 越大越要求分好
gamma = 0.0 #seqloss 越大越要求連續
#-0.001 / 1.0 / 1.0

@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=False) as tape:
        emb, pred_imp = model1(x)
        #loss1 = alpha*loss_object1(pred_imp) #phase1
        #pred_imp2 = tf.math.round(pred_imp)
        #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
        pred_imp2 = tf.math.round(pred_imp)
        loss1 = loss_object1(pred_imp) #有1-就是希望1越多，沒1-就是希望0越多1越少 #pahse2:alpha*loss_object1(pred_imp) ; phase1: alpha*(1-loss_object1(pred_imp))
        pred_cat = model2(emb,pred_imp2) #pahse1: pred_imp; phase2; pred_imp2
        loss2 = loss_object2(yc, pred_cat)
        loss3 = 1-seq_loss(pred_imp2)
        loss = alpha*loss1 + beta*loss2 + gamma*loss3
    trainable_variable = model1.trainable_variables
    trainable_variable.extend(model2.trainable_variables)
    gradients = tape.gradient(loss,trainable_variable)
    optimizer1.apply_gradients(zip(gradients,trainable_variable))
    
    train_loss(loss) #total_loss
    train_filloss(loss1)
    train_accloss(loss2)
    train_seqloss(loss3) #loss_seq
    train_accuracy(yc, pred_cat) #acc_rate
    ones = one_percentage(pred_imp2) #pred_imp2
    train_ones(ones) #ones_num
    
    
@tf.function
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)

    
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)


# def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
#     ori_word_li = []
#     for item in listOfValues:
#         key = item.numpy()
#         if key == 0:
#             continue
#         ori_word = dfOfElements.loc[key][0]
#         ori_word_li.append(ori_word)
#     return ori_word_li
ep = -1
@tf.function
def test_step_inf(x,yc,epoch):
    #ori_text = getKeysByValues(list(x))
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    #if ep != epoch:
        #ep = epoch
        #x = tf.dtypes.cast(x,dtype=tf.float32)
    pred_imp3 = tf.squeeze(pred_imp2)
    pred_imp3 = tf.dtypes.cast(pred_imp3,dtype=tf.int32)
    used_x = tf.math.multiply(x,pred_imp3)
        #word_reproduce1(x)
        #word_reproduce2(used_x)
        #used_text = getKeysByValues(list(used_x))
        #tf.print(used_x)
        #print(emb.numpy(),pred_imp, x,pred_imp3,used_x)
        #int_ori_text.append(x)
        #int_fil_text.append(used_x)
    
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
    return x, used_x

        ori_word
ID              
10996       mdbl
16260       fawc
12089    degussa
8803       woods
13796    hanging
...          ...
16258    rotting
10995       pods
2849       emery
30979  northerly
16259   onomichi

[30979 rows x 1 columns]


In [331]:
'''
#AIO
alpha = 0.1
beta = 1
gamma = 0
@tf.function
def train_step(x,yc):
    with tf.GradientTape(persistent=True) as tape: #persistent=True
        pred_imp,pred_round , pred_cat = model(x)
#         pred_cat = model(x)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
        loss1 = alpha*loss_object1(pred_imp)
        loss2 = beta*loss_object2(yc,pred_cat)
        loss = loss_object2(yc,pred_cat)
#     gradients = tape.gradient(loss, model.trainable_variables)
    grad1 = tape.gradient(loss1, model.trainable_variables)
    grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer1.apply_gradients(zip(gradients, model.trainable_variables))
    optimizer1.apply_gradients(zip(grad1, model.trainable_variables))
    optimizer2.apply_gradients(zip(grad2, model.trainable_variables))
#     with tf.GradientTape() as tape:
#         pred_imp , pred_cat = model(x)
#         loss2 = loss_object2(yc,pred_cat)
#         loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat)
#     grad2 = tape.gradient(loss2, model.trainable_variables)
#     optimizer2.apply_gradients(zip(grad2, model.trainable_variables))

    train_loss(loss)
    train_accuracy(yc, pred_cat)
    ones = one_percentage(pred_round)
    train_ones(ones)
    
@tf.function
def test_step(x,yc):
    pred_imp,pred_round, pred_cat = model(x)
#     pred_cat = model(x)
    t_loss = alpha*loss_object1(pred_imp) + loss_object2(yc,pred_cat) 
#     t_loss = loss_object2(yc,pred_cat)
    
    test_loss(t_loss)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_round)
    test_ones(t_ones)
'''    

In [97]:
EPOCHS = 2000
DateID = '2019111402'

saveP1 = './model/'+DateID+'/model1'
saveP2 = './model/'+DateID+'/model2'
train_loss_acc = []
train_loss_filter = []
train_loss_seq = []
train_weighted_loss = []
train_acc_rate = []
train_ones_num = []

test_loss_acc = []
test_loss_filter = []
test_loss_seq = []
test_weighted_loss = []
test_acc_rate = []
test_ones_num = []

int_ori_text =[]
int_fil_text = []
# signature_dict = {'att':model1.att}

gc.collect()
best_clf = 0.0
for epoch in range(EPOCHS):
    for text, labels in train_ds:
        train_step(text, labels)

    for test_text, test_labels in valid_ds:
        ori,fil = test_step_inf(test_text, test_labels,epoch) #with restore words
        #print(k2)
        #test_step(test_text, test_labels) #original
    int_ori_text.append(ori.numpy())
    int_fil_text.append(fil.numpy())
    
    template = 'Epoch {}, Total Loss: {}, Clf Loss: {}, Filter Loss: {}, Seq Loss: {}, Accuracy Rate: {:5.2f}%, Ones Portion: {}, \
            Test_Total_Loss: {}, Test_Clf_Loss: {}, Test_Filter_Loss: {}, TEST_Seq_Loss: {}, Test_Accuracy_Rate: {:5.2f}%, Test_Ones_Portion: {}'
    print(template.format(epoch+1,train_loss.result(),
                          train_accloss.result(),train_filloss.result(),train_seqloss.result(),
                        train_accuracy.result()*100,train_ones.result(),
                        test_loss.result(),
                        test_accloss.result(),test_filloss.result(),test_seqloss.result(),
                        test_accuracy.result()*100,test_ones.result(),
                        ))
    #int_ori_text.append(word_reproduce1.result().numpy())
    #int_fil_text.append(word_reproduce2.result().numpy())
    train_loss_acc.append( train_accloss.result().numpy())
    train_loss_filter.append( train_filloss.result().numpy())
    train_loss_seq.append( train_seqloss.result().numpy())
    train_weighted_loss.append( train_loss.result().numpy())
    train_acc_rate.append( train_accuracy.result().numpy())
    train_ones_num.append( train_ones.result().numpy())
    
    test_loss_acc.append( test_accloss.result().numpy())
    test_loss_filter.append( test_filloss.result().numpy())
    test_loss_seq.append( test_seqloss.result().numpy())
    test_weighted_loss.append( test_loss.result().numpy())
    test_acc_rate.append( test_accuracy.result().numpy())
    test_ones_num.append( test_ones.result().numpy())
    if best_clf<=test_accuracy.result()*100:
        #tf.saved_model.save(model1,saveP1+'_all')
        #model1.save(saveP1,save_format='h5')
        #model2.save(saveP2,save_format='h5')
        #tf.saved_model.save(model2,saveP2+'_all')
        model1.save_weights(saveP1,save_format='tf')
        model2.save_weights(saveP2,save_format='tf')
        best_clf = test_accuracy.result()*100
        print('===MODEL WEIGHTS SAVED===',saveP1,saveP2)
    # Reset the metrics for the next epoch
    #word_reproduce1.reset_states()
    #word_reproduce2.reset_states()
    train_loss.reset_states()
    train_accloss.reset_states()
    train_filloss.reset_states()
    train_seqloss.reset_states()
    train_ones.reset_states()
    train_accuracy.reset_states()
    
    test_loss.reset_states()
    test_accloss.reset_states()
    test_filloss.reset_states()
    test_seqloss.reset_states()
    test_ones.reset_states()
    test_accuracy.reset_states()

Epoch 1, Total Loss: 3.3594601154327393, Clf Loss: 3.3594589233398438, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 34.79%, Ones Portion: 1.0,             Test_Total_Loss: 2.555807590484619, Test_Clf_Loss: 2.5558063983917236, Test_Filter_Loss: 0.6000000238418579, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/2019111402/model1 ./model/2019111402/model2
Epoch 2, Total Loss: 2.3976430892944336, Clf Loss: 2.397641658782959, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 37.72%, Ones Portion: 1.0,             Test_Total_Loss: 2.366929292678833, Test_Clf_Loss: 2.3669278621673584, Test_Filter_Loss: 0.6000000238418579, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 37.17%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/2019111402/model1 ./model/2019111402/model2
Epoch 3, Total Loss: 2.3459160327911377, Clf Loss: 2.345914840698242, Filter Loss: 0.5999999642372131, Seq Loss: 0.0, Accuracy Rate: 37.72%

Epoch 23, Total Loss: 1.3744086027145386, Clf Loss: 1.3744075298309326, Filter Loss: 0.5999988317489624, Seq Loss: 0.0, Accuracy Rate: 65.61%, Ones Portion: 1.0,             Test_Total_Loss: 1.757117509841919, Test_Clf_Loss: 1.7571160793304443, Test_Filter_Loss: 0.5999990701675415, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 57.26%, Test_Ones_Portion: 1.0
Epoch 24, Total Loss: 1.352891206741333, Clf Loss: 1.352890133857727, Filter Loss: 0.599998950958252, Seq Loss: 0.0, Accuracy Rate: 66.09%, Ones Portion: 1.0,             Test_Total_Loss: 1.7720295190811157, Test_Clf_Loss: 1.7720284461975098, Test_Filter_Loss: 0.5999990701675415, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 57.45%, Test_Ones_Portion: 1.0
===MODEL WEIGHTS SAVED=== ./model/2019111402/model1 ./model/2019111402/model2
Epoch 25, Total Loss: 1.3466858863830566, Clf Loss: 1.3466848134994507, Filter Loss: 0.599998950958252, Seq Loss: 0.0, Accuracy Rate: 66.26%, Ones Portion: 1.0,             Test_Total_Loss: 1.7643623352050781, Test_C

Epoch 46, Total Loss: 1.1371980905532837, Clf Loss: 1.1371968984603882, Filter Loss: 0.5999988913536072, Seq Loss: 0.0, Accuracy Rate: 69.39%, Ones Portion: 1.0,             Test_Total_Loss: 1.9570660591125488, Test_Clf_Loss: 1.9570651054382324, Test_Filter_Loss: 0.5999976992607117, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 55.66%, Test_Ones_Portion: 1.0
Epoch 47, Total Loss: 1.1197469234466553, Clf Loss: 1.1197458505630493, Filter Loss: 0.599998950958252, Seq Loss: 0.0, Accuracy Rate: 70.00%, Ones Portion: 1.0,             Test_Total_Loss: 1.9447031021118164, Test_Clf_Loss: 1.944701910018921, Test_Filter_Loss: 0.5999979376792908, TEST_Seq_Loss: 0.0, Test_Accuracy_Rate: 56.29%, Test_Ones_Portion: 1.0
Epoch 48, Total Loss: 1.1127867698669434, Clf Loss: 1.1127853393554688, Filter Loss: 0.5999990105628967, Seq Loss: 0.0, Accuracy Rate: 69.69%, Ones Portion: 1.0,             Test_Total_Loss: 1.96897554397583, Test_Clf_Loss: 1.9689741134643555, Test_Filter_Loss: 0.5999978184700012, TEST_Seq_L

Epoch 68, Total Loss: 0.9100519418716431, Clf Loss: 0.9100509881973267, Filter Loss: 0.5999977588653564, Seq Loss: 8.897597581380978e-06, Accuracy Rate: 77.13%, Ones Portion: 0.999994158744812,             Test_Total_Loss: 2.06472110748291, Test_Clf_Loss: 2.0647196769714355, Test_Filter_Loss: 0.5999939441680908, TEST_Seq_Loss: 1.1486165931273717e-05, Test_Accuracy_Rate: 58.23%, Test_Ones_Portion: 0.9999922513961792
===MODEL WEIGHTS SAVED=== ./model/2019111402/model1 ./model/2019111402/model2
Epoch 69, Total Loss: 0.9061325192451477, Clf Loss: 0.9061317443847656, Filter Loss: 0.5999977588653564, Seq Loss: 8.01086389401462e-06, Accuracy Rate: 77.49%, Ones Portion: 0.999994695186615,             Test_Total_Loss: 2.108489513397217, Test_Clf_Loss: 2.108488082885742, Test_Filter_Loss: 0.5999935269355774, TEST_Seq_Loss: 1.1486165931273717e-05, Test_Accuracy_Rate: 56.92%, Test_Ones_Portion: 0.9999922513961792
Epoch 70, Total Loss: 0.9184343814849854, Clf Loss: 0.9184331297874451, Filter Loss: 

Epoch 88, Total Loss: 0.7853603959083557, Clf Loss: 0.7853592038154602, Filter Loss: 0.5999942421913147, Seq Loss: 6.30378708592616e-05, Accuracy Rate: 80.31%, Ones Portion: 0.9999580979347229,             Test_Total_Loss: 2.204728126525879, Test_Clf_Loss: 2.2047269344329834, Test_Filter_Loss: 0.599986732006073, TEST_Seq_Loss: 0.00015848524344619364, Test_Accuracy_Rate: 57.45%, Test_Ones_Portion: 0.999890148639679
Epoch 89, Total Loss: 0.7728459239006042, Clf Loss: 0.7728447318077087, Filter Loss: 0.5999941825866699, Seq Loss: 6.214746827026829e-05, Accuracy Rate: 80.15%, Ones Portion: 0.9999589323997498,             Test_Total_Loss: 2.23230242729187, Test_Clf_Loss: 2.2323012351989746, Test_Filter_Loss: 0.5999866127967834, TEST_Seq_Loss: 0.00015848524344619364, Test_Accuracy_Rate: 56.87%, Test_Ones_Portion: 0.999890148639679
Epoch 90, Total Loss: 0.7655916213989258, Clf Loss: 0.765590250492096, Filter Loss: 0.5999941825866699, Seq Loss: 6.213921733433381e-05, Accuracy Rate: 80.57%, One

...

Epoch 1908, Total Loss: 0.9198853969573975, Clf Loss: 0.9198853969573975, Filter Loss: 0.00019979284843429923, Seq Loss: 0.9999898076057434, Accuracy Rate: 77.05%, Ones Portion: 0.00010866474622162059,             Test_Total_Loss: 2.896026849746704, Test_Clf_Loss: 2.896026849746704, Test_Filter_Loss: 0.00024144469352904707, TEST_Seq_Loss: 0.9999890327453613, Test_Accuracy_Rate: 58.62%, Test_Ones_Portion: 0.00012077832070644945
Epoch 1909, Total Loss: 0.9029724597930908, Clf Loss: 0.9029724597930908, Filter Loss: 0.0001954416948137805, Seq Loss: 0.9999902844429016, Accuracy Rate: 77.13%, Ones Portion: 0.00010652849596226588,             Test_Total_Loss: 2.8919389247894287, Test_Clf_Loss: 2.8919389247894287, Test_Filter_Loss: 0.00024034630041569471, TEST_Seq_Loss: 0.9999890327453613, Test_Accuracy_Rate: 58.81%, Test_Ones_Portion: 0.00012034521205350757
Epoch 1910, Total Loss: 0.8921778798103333, Clf Loss: 0.8921778798103333, Filter Loss: 0.0001978822547243908, Seq Loss: 0.999990403652191

Epoch 1928, Total Loss: 0.8668959140777588, Clf Loss: 0.8668959140777588, Filter Loss: 0.00019969098502770066, Seq Loss: 0.9999902844429016, Accuracy Rate: 77.86%, Ones Portion: 0.00010719006240833551,             Test_Total_Loss: 2.9641611576080322, Test_Clf_Loss: 2.9641611576080322, Test_Filter_Loss: 0.0002379172801738605, TEST_Seq_Loss: 0.9999890327453613, Test_Accuracy_Rate: 58.37%, Test_Ones_Portion: 0.00012162897473899648
Epoch 1929, Total Loss: 0.8680145740509033, Clf Loss: 0.8680145740509033, Filter Loss: 0.000198232926777564, Seq Loss: 0.9999904036521912, Accuracy Rate: 77.98%, Ones Portion: 0.00010635390208335593,             Test_Total_Loss: 2.9800467491149902, Test_Clf_Loss: 2.9800467491149902, Test_Filter_Loss: 0.00023911765310913324, TEST_Seq_Loss: 0.9999889135360718, Test_Accuracy_Rate: 58.42%, Test_Ones_Portion: 0.00012220982171129435
Epoch 1930, Total Loss: 0.8634305596351624, Clf Loss: 0.8634305596351624, Filter Loss: 0.00019936395983677357, Seq Loss: 0.99999016523361

Epoch 1948, Total Loss: 0.8414421081542969, Clf Loss: 0.8414421081542969, Filter Loss: 0.00020201111328788102, Seq Loss: 0.9999894499778748, Accuracy Rate: 78.43%, Ones Portion: 0.00011002711835317314,             Test_Total_Loss: 2.995427131652832, Test_Clf_Loss: 2.995427131652832, Test_Filter_Loss: 0.0002448796876706183, TEST_Seq_Loss: 0.9999884366989136, Test_Accuracy_Rate: 58.86%, Test_Ones_Portion: 0.00012497817806433886
Epoch 1949, Total Loss: 0.842610239982605, Clf Loss: 0.842610239982605, Filter Loss: 0.00020251887326594442, Seq Loss: 0.999989926815033, Accuracy Rate: 78.46%, Ones Portion: 0.00011020036617992446,             Test_Total_Loss: 2.9818296432495117, Test_Clf_Loss: 2.9818296432495117, Test_Filter_Loss: 0.00024328452127519995, TEST_Seq_Loss: 0.9999885559082031, Test_Accuracy_Rate: 59.10%, Test_Ones_Portion: 0.0001246347965206951
Epoch 1950, Total Loss: 0.8431650996208191, Clf Loss: 0.8431650996208191, Filter Loss: 0.00020072443294338882, Seq Loss: 0.9999900460243225, 

Epoch 1968, Total Loss: 0.8482761383056641, Clf Loss: 0.8482761383056641, Filter Loss: 0.00019367890490684658, Seq Loss: 0.9999904632568359, Accuracy Rate: 78.17%, Ones Portion: 0.0001066267432179302,             Test_Total_Loss: 3.0098602771759033, Test_Clf_Loss: 3.0098602771759033, Test_Filter_Loss: 0.00023605988826602697, TEST_Seq_Loss: 0.9999890327453613, Test_Accuracy_Rate: 59.15%, Test_Ones_Portion: 0.000122151366667822
Epoch 1969, Total Loss: 0.8440393209457397, Clf Loss: 0.8440393209457397, Filter Loss: 0.0001961829693755135, Seq Loss: 0.9999904036521912, Accuracy Rate: 78.41%, Ones Portion: 0.00010760729492176324,             Test_Total_Loss: 3.0122547149658203, Test_Clf_Loss: 3.0122547149658203, Test_Filter_Loss: 0.0002355584583710879, TEST_Seq_Loss: 0.9999890327453613, Test_Accuracy_Rate: 59.34%, Test_Ones_Portion: 0.00012209195119794458
Epoch 1970, Total Loss: 0.8414967656135559, Clf Loss: 0.8414967656135559, Filter Loss: 0.00019469656399451196, Seq Loss: 0.9999902844429016

Epoch 1988, Total Loss: 0.8552099466323853, Clf Loss: 0.8552099466323853, Filter Loss: 0.00019006965158041567, Seq Loss: 0.9999905824661255, Accuracy Rate: 78.16%, Ones Portion: 0.00010551192826824263,             Test_Total_Loss: 3.099846124649048, Test_Clf_Loss: 3.099846124649048, Test_Filter_Loss: 0.00022726820316165686, TEST_Seq_Loss: 0.9999893307685852, Test_Accuracy_Rate: 58.91%, Test_Ones_Portion: 0.0001196421217173338
Epoch 1989, Total Loss: 0.8384340405464172, Clf Loss: 0.8384340405464172, Filter Loss: 0.00019032106501981616, Seq Loss: 0.9999909400939941, Accuracy Rate: 78.32%, Ones Portion: 0.00010581820242805406,             Test_Total_Loss: 3.0772705078125, Test_Clf_Loss: 3.0772705078125, Test_Filter_Loss: 0.0002279468608321622, TEST_Seq_Loss: 0.9999892115592957, Test_Accuracy_Rate: 58.81%, Test_Ones_Portion: 0.00012091993994545192
Epoch 1990, Total Loss: 0.8348019123077393, Clf Loss: 0.8348019123077393, Filter Loss: 0.00019303345470689237, Seq Loss: 0.9999904632568359, Acc

In [107]:
int_fil_text[-100]

array([[  1,   2,  71, ...,   0,   0,   0],
       [  1,   2, 155, ...,   0,   0,   0],
       [  1,  56,   2, ...,   0,   0,   0],
       ...,
       [  1,  53, 119, ...,   0,   0,   0],
       [  1,  53,   0, ...,   0,   0,   0],
       [  1,  53,  46, ...,   0,   0,   0]], dtype=int32)

### Store process

In [98]:
save_dir = './results/'+DateID+'/'
if not os.path.isdir(save_dir):
    os.makedirs(save_dir, exist_ok=True)
saveR = save_dir + 'losses_metrics.xlsx'
data = {'train total loss':train_weighted_loss, 'train acc loss':train_loss_acc,
        'train filter loss':train_loss_filter,'train seq loss':train_loss_seq,
        'train acc rate':train_acc_rate, 'train ones num':train_ones_num,
        'test total loss':test_weighted_loss, 'test acc loss':test_loss_acc,
        'test filter loss':test_loss_filter, 'test seq loss': test_loss_seq,
        'test acc rate':test_acc_rate, 'test ones num':test_ones_num
       }
df = pd.DataFrame(data)
df.to_excel(saveR)
print(saveR)
df

./results/2019111402/losses_metrics.xlsx


,train total loss,train acc loss,train filter loss,train seq loss,train acc rate,train ones num,test total loss,test acc loss,test filter loss,test seq loss,test acc rate,test ones num
0,3.359460,3.359459,0.600000,0.000000,0.347942,1.000000,2.555808,2.555806,0.600000,0.000000,0.371733,1.000000
1,2.397643,2.397642,0.600000,0.000000,0.377240,1.000000,2.366929,2.366928,0.600000,0.000000,0.371733,1.000000
2,2.345916,2.345915,0.600000,0.000000,0.377240,1.000000,2.359270,2.359269,0.600000,0.000000,0.371733,1.000000
3,2.339791,2.339790,0.600000,0.000000,0.377240,1.000000,2.358807,2.358806,0.600000,0.000000,0.371733,1.000000
4,2.338617,2.338616,0.600000,0.000000,0.377240,1.000000,2.354495,2.354494,0.600000,0.000000,0.371733,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,0.826181,0.826181,0.000192,0.999990,0.784867,0.000107,3.095845,3.095845,0.000226,0.999989,0.588577,0.000121
1996,0.830815,0.830815,0.000188,0.999991,0.785714,0.000106,3.072968,3.072968,0.000226,0.999989,0.594385,0.000121
1997,0.830443,0.830443,0.000188,0.999991,0.788499,0.000106,3.110749,3.110749,0.000227,0.999989,0.591481,0.000120
1998,0.838554,0.838554,0.000190,0.999990,0.784019,0.000106,3.060227,3.060227,0.000226,0.999989,0.590997,0.000120


## Inference
* Case study

In [119]:
word_idx_pd = pd.DataFrame(data=word_index,index=["ID"]).T
word_idx_pd['ori_word'] = word_idx_pd.index 
word_idx_pd = word_idx_pd.set_index(["ID"])
print(word_idx_pd)

def getKeysByValues(listOfValues,dfOfElements=word_idx_pd):
    ori_word_li = []
    for item in listOfValues:
        key = item
#         print(key)
        if key == 0:
            continue
        ori_word = dfOfElements.loc[key][0]
        ori_word_li.append(ori_word)
    return ori_word_li

        ori_word
ID              
10996       mdbl
16260       fawc
12089    degussa
8803       woods
13796    hanging
...          ...
16258    rotting
10995       pods
2849       emery
30979  northerly
16259   onomichi

[30979 rows x 1 columns]


In [132]:
word_fil_text = []
word_ori_text = []
from tqdm import tqdm
for text in tqdm(int_ori_text): #text = one epoch
    temp = []
    for t in text: #t = one sample
        temp.append(getKeysByValues(list(t)))
    word_ori_text.append(temp)
for text in tqdm(int_fil_text):
    temp = []
    for t in text:
        temp.append(getKeysByValues(list(t)))
    word_fil_text.append(temp)

                        

100%|██████████| 2000/2000 [02:06<00:00, 15.80it/s]


In [138]:
word_fil_text[-1500]

[['the',
  'of',
  'profit',
  'mln',
  'at',
  'buy',
  '3',
  'buy',
  'said',
  'in',
  'paulo',
  'could',
  '6',
  'paulo',
  'definitive',
  'subject',
  'in',
  'still',
  'said',
  'in',
  'corp',
  'central',
  'a',
  'in',
  'still',
  'of',
  'willing',
  'he',
  'boost',
  'from',
  '4',
  'tonnes',
  'said',
  'of',
  'oper',
  '10',
  'it',
  'expects',
  'income',
  '0',
  'said',
  'paulo',
  'oper',
  '10',
  'of',
  'mln',
  'pct',
  'dlrs'],
 ['the',
  'of',
  '16',
  'is',
  'said',
  '187',
  '3',
  "o'reilly",
  'drop',
  'warrant',
  'shell',
  'bank',
  'sales',
  'of',
  '9',
  'for',
  '30',
  'australia',
  'said',
  'in',
  'revs',
  '2',
  'economy',
  'conference',
  'and',
  'in',
  '187',
  'national',
  "turkey's",
  'further',
  'particularly',
  'hostile',
  '101',
  'national',
  'activity',
  'it',
  'stake',
  'coast',
  'was',
  'were',
  'new',
  'vs',
  '35',
  'it',
  'are',
  '2',
  'office',
  'rather',
  'its',
  'have',
  '2',
  'essential'

In [ ]:
def test_step(x,yc):
    emb, pred_imp = model1(x)
    #loss1 = alpha*loss_object1(pred_imp) #phase1
    #pred_imp2 = tf.math.round(pred_imp)
    #pred_imp3 = tf.clip_by_value(pred_imp,clip_value_max=1,clip_value_min=0)
    pred_imp2 = tf.math.round(pred_imp)
    loss1 = loss_object1(pred_imp) #phase2
    pred_cat = model2(emb,pred_imp2) #phase1: pred_imp ; phase2:pred_imp2
    loss2 = loss_object2(yc, pred_cat)
    loss3 = 1-seq_loss(pred_imp2)
    #t_loss = loss1 + loss2
    t_loss = alpha*loss1 + beta*loss2 + gamma*loss3
    
    test_loss(t_loss)
    test_filloss(loss1)
    test_accloss(loss2)
    test_seqloss(loss3)
    test_accuracy(yc, pred_cat)
    t_ones = one_percentage(pred_imp2) #pred_imp2
    test_ones(t_ones)
for test_text, test_labels in valid_ds:
    test_step(test_text, test_labels)

In [7]:
for test_text, test_labels in valid_ds:
    print(test_text)
    print(test_labels)
    break
#     test_step(test_text, test_labels)

NameError: name 'valid_ds' is not defined

In [11]:
convert_li = getKeysByValues(testX[0])
convert_li

['the',
 'in',
 'compared',
 'analysts',
 'etl',
 'at',
 'may',
 'said',
 'si',
 'offshore',
 'a',
 'but',
 'a',
 'ohio',
 'and',
 'publicly',
 'its',
 'sent',
 '380',
 'plan',
 'bank',
 'mln',
 '1',
 '756',
 'not',
 'note',
 'was',
 'finding',
 'eased',
 '80',
 'in',
 'compared',
 'analysts',
 'as',
 'main',
 'and',
 'origins',
 'be',
 'in',
 'borrowers',
 'said',
 'in',
 'hearings',
 'said',
 'becoming',
 '3',
 'as',
 'of',
 'was',
 'in',
 'worsen',
 'analysts',
 'that',
 'of',
 'offshore',
 'flight',
 'for',
 'during',
 'time',
 'official',
 'mln',
 'a',
 'for',
 'capital',
 'in',
 'par',
 'said',
 'in',
 'worsen',
 'sought',
 'bank',
 'sales',
 'commodity',
 'common',
 'above',
 'a',
 'merchants',
 'amounted',
 'already',
 'in',
 'capital',
 'mln',
 'in',
 'compared',
 'analysts',
 '3',
 'in',
 'worsen',
 '87',
 '41',
 'for',
 'hearings',
 'said',
 'becoming',
 'importers',
 'offshore',
 'flight',
 'be',
 'in',
 'has',
 'would',
 '24',
 '32',
 'firms',
 'of',
 'a',
 'only',
 'cattl

In [12]:
reuters.get_word_index()

{'mdbl': 10996,
 'fawc': 16260,
 'degussa': 12089,
 'woods': 8803,
 'hanging': 13796,
 'localized': 20672,
 'sation': 20673,
 'chanthaburi': 20675,
 'refunding': 10997,
 'hermann': 8804,
 'passsengers': 20676,
 'stipulate': 20677,
 'heublein': 8352,
 'screaming': 20713,
 'tcby': 16261,
 'four': 185,
 'grains': 1642,
 'broiler': 20680,
 'wooden': 12090,
 'wednesday': 1220,
 'highveld': 13797,
 'duffour': 7593,
 '0053': 20681,
 'elections': 3914,
 '270': 2563,
 '271': 3551,
 '272': 5113,
 '273': 3552,
 '274': 3400,
 'rudman': 7975,
 '276': 3401,
 '277': 3478,
 '278': 3632,
 '279': 4309,
 'dormancy': 9381,
 'errors': 7247,
 'deferred': 3086,
 'sptnd': 20683,
 'cooking': 8805,
 'stratabit': 20684,
 'designing': 16262,
 'metalurgicos': 20685,
 'databank': 13798,
 '300er': 20686,
 'shocks': 20687,
 'nawg': 7972,
 'tnta': 20688,
 'perforations': 20689,
 'affiliates': 2891,
 '27p': 20690,
 'ching': 16263,
 'china': 595,
 'wagyu': 16264,
 'affiliated': 3189,
 'chino': 16265,
 'chinh': 16266,
 '

,ori_word
ID,
10996,mdbl
16260,fawc
12089,degussa
8803,woods
13796,hanging
...,...
16258,rotting
10995,pods
2849,emery


In [1047]:
test_text[0][0].numpy()

1

In [1049]:
word_idx_pd.loc[test_text[0][0].numpy()][0]

'the'

In [1036]:
word_idx_pd

,ID
mdbl,10996
fawc,16260
degussa,12089
woods,8803
hanging,13796
...,...
rotting,16258
pods,10995
emery,2849
northerly,30979


In [714]:
np.array(train_weighted_loss[0])

array(None, dtype=object)

In [732]:
kk = np.array(test_accloss.result())
kk

array(2.3583207, dtype=float32)

In [735]:
test_accloss.result().numpy()

2.3583207

In [523]:
# model1.save('./model/emb_layer') #tf.keras.models.load_model
tf.saved_model.save(model1,'./model/emb_layer')
model3 = tf.keras.models.load_model('./model/emb_layer')

TypeError: unsupported callable

In [524]:
model3 = model1.load_weights('./model/emb_layer_weight')

AttributeError: 'CheckpointLoadStatus' object has no attribute 'summary'

In [527]:
model1.emb.set_weights(model3.emb.get_weights())

In [526]:
model3 = model1
model3.load_weights('./model/emb_layer_weight')
model3.summary()

Model: "base_model_1_58"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking_58 (Masking)         multiple                  0         
_________________________________________________________________
glove_emb (Embedding)        multiple                  2138112   
_________________________________________________________________


ValueError: You tried to call `count_params` on common_extract, but the layer isn't built. You can build it manually via: `common_extract.build(batch_input_shape)`.

* 在某些參數的情況下0 1是train不起來的但是weight可以
    * alpha = 0.0, beta=0.6
    * init_w = tensorflow.keras.initializers.Constant(value=0.9), init_b = tensorflow.keras.initializers.Constant(value=0.7)
* 0/1 with emb比較容易overfit。如果是weight的比較沒那麼嚴重

* 不用sigmoid或是hard_sigmoid。改良relu + linear，並拆成兩個model，把round前面多加上clip
    * sigmoid中間的變化太快(一瞬間就會掉到0或是1)，改成relu在>0~無限大(linear為了還在0~1)再去clip再round，可以看到每個epoch的變化
* 1st phase的beta一定要>=0.6否則不會動，ones#都會是0
    * beta=1 (放0 1進去): weight init設成1也沒用，但把bias設成1就會一開始都是ones#=1了。weight=1 bias=0.2都匯市0，bias=0.3會是0.87。0.6/0.3都是0。0.8/0.3差不多是0.5(但是train很慢acc進步很慢就是了)。0.5/0.5是從0.01開始往上升 (0 1放進去會比較難train是因為它的變化量太大，一下就是有或沒有，所以clf可能學不好，但如果是weight每次gradient進步的都是一小點就會比較容易上升)
    * beta=0.6 (放0 1進去): 0.8/0.5都是0。0.9/0.8 從0.9一直到0。0.9/0.6差不多是從0.5但又有時候到0.7都是0.0(很難train，Nadam換個opt有時候沒用。EX變成adam 0.9/0.8才有0.96開始但如果0.9/0.75變成0開始。Rmsprop 0.9/0.8又是從0.95開始往下)。但如果都改成傳入weight就都沒問題。ones一開始大概0.5 weight平均，也不會卡住
* 建議: 先訓練embedding weight matrix，但是要看goal是要怎樣的matrix
* 其實他不管幾%都會train得很好，除非固定embedding，或是用更弱的clf
* 若設定alpha，就像是regularizer term (penalty)，設越大drop越多
* 一開始先很多ones，再越來越少個

* 同一個opt若加入transform就會train不起來
* 兩個不同的opt加入transform也會train不起來 (persistent、non-persis都不行)，且與BN無關